In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow import keras
from tensorflow.keras import layers, models, Input

In [3]:
#有向無環圖(Directed Acyclic Graphs)
#Inception神經網路由一堆Inception模組(module)組成
#基本形式的Inception模組有3、4個分支，從1*1卷積開始，然後是3*3卷積，最後是結果特徵(張量)的串接，這樣設計有助於神經網路分別學習資料的channel特徵和空間特徵
#複雜的Inception模組版本則涉及池化(pooling)的操作，或使用不同的空間卷積大小，以及沒有空間卷積的分支(僅1*1卷積:將輸入張量的channel資訊混在一起計算成特徵圖，不會混合到空間，也稱"逐點卷積")

In [4]:
x = Input(batch_shape=(1000, 28, 28, 256)) #定義4D向量
print(x.shape) #shape=(1000,28,28,256)

#======================================================================================================#
branch_a = layers.Conv2D(64, 1, activation='relu', strides=2)(x) #進行1/2採樣
print(branch_a.shape) #shape=(1000,14,14,64)

#======================================================================================================#
branch_b = layers.Conv2D(128, 1, activation='relu')(x) #未進行採樣
print(branch_b.shape) #shape=(1000,28,28,128)

branch_b = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_b) #進行1/2採樣
print(branch_b.shape) #shape=(1000,14,14,128)

#======================================================================================================#
branch_c = layers.AveragePooling2D(3, strides=2, padding='same')(x) #採樣發生在平均池化層中
print(branch_c.shape) #shape=(1000,14,14,256)

branch_c = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_c)
print(branch_c.shape) #shape=(1000,14,14,128)

#======================================================================================================#
branch_d = layers.Conv2D(128, 1, activation='relu')(x)
print(branch_d.shape) #shape=(1000,28,28,128)

branch_d = layers.Conv2D(128, 3 , activation='relu', padding='same')(branch_d)
print(branch_d.shape) #shape=(1000,28,28,128)

branch_d = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_d) #進行1/2採樣
print(branch_d.shape) #shape=(1000,14,14,128)

(1000, 28, 28, 256)
(1000, 14, 14, 64)
(1000, 28, 28, 128)
(1000, 14, 14, 128)
(1000, 14, 14, 256)
(1000, 14, 14, 128)
(1000, 28, 28, 128)
(1000, 28, 28, 128)
(1000, 14, 14, 128)


2022-08-27 06:55:23.646970: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
#串接分支輸出以取得模組輸出
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)
print(output.shape)

(1000, 14, 14, 448)


In [6]:
#每個分支皆有一個以相同步長(strides=2)進行採樣的層，這是為了保持所有分支輸出張量大小相同所必需的設定，以便最後可以串接每個分支的結果，但各分支的filters值不一定相同

In [7]:
#Xception = extreme Inception(極端的Inception)
#它採用將channel特徵和空間特徵的學習分離成邏輯極值的概念，並用深度可分離卷積層替換Inception模組
#該卷積層由深度卷積組成(將輸入依channel切割，並個別處理空間卷積)，串接空間卷積的結果後進行逐點卷積(1*1卷積)
#其中空間特徵、channel特徵明確，有效地完全分離，因為能更有效地使用模型參數，因此在大型資料集上取得更好的執行性能與準確度

In [8]:
#殘差連接(Residual connections)
#神經網路架構中常見的圖形結構神經網路元件，包括剛剛提到的Xception
#解決了大規模(多層)深度學習模型的兩個常見問題: 梯度消失、轉換瓶頸
#殘差連接主要將上游層的輸出作為下游層的輸入，進而有效地在序列式神經網路中建立捷徑
#將較早的啟動函數輸出張量與後面的啟動函數輸出張量相加，若張量的shape大小相同，則可直接相加；若大小不同，則可使用線性轉換將較早的張量shape調整為下游張量的shape
#線性轉換: T:矩陣，可加性: T(u + v) = T(u) + T(v)，齊次性: aT(u) = T(au)[a為任意純量]
#神經網路中啟動函數結果(張量)也可透過線性轉換改變其shape大小，ex:使用沒有啟動函數的Dense層，卷積特徵圖則是使用沒有啟動函數的1*1卷積

In [9]:
#啟動函數輸出張量shape大小相同的殘差連接
x = Input(batch_shape=(1000, 32, 32, 128)) #定義4D張量
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x) #y.shape=(1000,32,32,128)
z = layers.Conv2D(128, 3, activation='relu', padding='same')(y) #z.shape=(1000,32,32,128)
op = layers.add([z, x]) #x與z的shape皆等於(1000,32,32,128)，可直接連接
print(op.shape)

(1000, 32, 32, 128)


In [10]:
#啟動函數輸出張量shape大小不同時，透過線性轉換，改變張量的shape，使其大小相同，再進行殘差連接
x = Input(batch_shape=(1000, 32, 32, 256)) #建立4D輸入張量
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
z = layers.Conv2D(128, 3 ,activation='relu', padding='same')(y)
print(z.shape) #shape=(1000,32,32,128)

t = layers.MaxPooling2D(2, strides=2)(z) #步長=2代表進行1/2的縮小採樣，張量大小減半(特徵圖的長寬)
print(t.shape) #shape=(1000,16,16,128)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x) #線性轉換: 對張量x使用步長=2的1*1卷積，進行1/2縮小採樣，並將channel降低成與張量t相同的128
print(residual.shape) #經過線性轉換後的殘差張量shape=(1000,16,16,128)

op = layers.add([t, residual]) #將殘差張量residual加回輸出特徵張量t
print(op.shape) #shape=(1000,16,16,128)

(1000, 32, 32, 128)
(1000, 16, 16, 128)
(1000, 16, 16, 128)
(1000, 16, 16, 128)
